In [ ]:
import numpy as np

# Базовый класс, задающий распределение непрерывной случайной величины
class Distribution:
    # возвращает sample(массив выборок) размера size
    def __call__(self, size = 1):
        return np.zeros(size)

    # подстраивает параметры распределения под заданные матожидание и дисперсию
    def fit_to_moments(self, mu : float, sigma : float):
        pass

# Равномерное распределение
class UniformDistribution(Distribution):
    def __init__(self, low = -1.0, high = 1.0):
        self.low = low
        self.high = high

    def fit_to_moments(self, mu: float, sigma: float):
        half_size = sigma / (4 * np.sqrt(3))
        self.low = mu - half_size
        self.high = mu + half_size

    def __call__(self, size = 1):
        return np.random.uniform(self.low, self.high, size)

# Нормальное распределение
class NormalDistribution(Distribution):
    def __init__(self, mu = 1.0, sigma = 1.0):
        self.mu = mu
        self.sigma = sigma

    def fit_to_moments(self, mu: float, sigma: float):
        self.mu = mu
        self.sigma = sigma

    def __call__(self, size = 1):
        return np.random.normal(self.mu, self.sigma, size)

# Экспоненциальное распределение: rho(x) = L * e^(-L * x) на [0, +inf)
# Частный случай Gamma-распределения при a = 1, b = 1/L
class ExpDistribution(Distribution):
    def __init__(self, L = 1.0):
        self.L = L

    # игнорирует предписанное матожидание
    def fit_to_moments(self, mu: float, sigma: float):
        self.L = 1/(sigma * sigma)

    def __call__(self, size = 1):
        return np.random.exponential(self.L, size)

# Beta-распределение: rho(x) = x^(a-1) * (1-x)^(b-1) / Beta(a, b) на [0, 1]
class BetaDistribution(Distribution):
    def __init__(self, alpha = 1.0, beta = 1.0):
        self.alpha = alpha
        self.beta = beta

    def __call__(self, size = 1):
        return np.random.beta(self.alpha, self.beta, size)

# Gamma-распределение: rho(x) = x^(a-1) * e^(-x/b) / (b^a * Gamma(a)) на [0, +inf)
class GammaDistribution(Distribution):
    def __init__(self, alpha = 0.0, beta = 1.0):
        self.alpha = alpha
        self.beta = beta

    def __call__(self, size = 1):
        return np.random.gamma(self.alpha, self.beta, size)

# Численно находит плотности распределения в заданных точках
def get_density(sample : np.ndarray, k : int):
    N = len(sample)
    # Матрица расстояний
    distances = np.abs(sample[:, np.newaxis] - sample[np.newaxis, :])
    sorted_distances = np.sort(distances, axis=1)
    # расстояние до k-ой ближайшей точки
    epsilon_k = sorted_distances[:, k]
    rho = k / (N * 2 * epsilon_k)
    return rho

# находит дифференциальную энтропию как -E[log(rho(x))] методом Монте-Карло, учитывая k-го соседа для вычисления плотности
def h(sample : np.ndarray, k : int):
    return -float(np.sum(np.log2(get_density(sample, k)))) / len(sample)

# Y = C + N, N ~ Norm(0, sigma^2)
class AWGNChannel:
    def __init__(self, sigma : float):
        self.N = NormalDistribution(0, sigma)
        self.sigma = sigma

    # вычисление взаимной информации I(C,Y)
    def get_I(self, C : Distribution):
        # монте карло
        pass

    def get_Y(self, C : Distribution, size = 1):
        return C(size) + self.N(size)

sigma = 10
mu = 0
dist = NormalDistribution(mu, sigma)
print(h(dist(10000), 200))
dist = UniformDistribution()
dist.fit_to_moments(mu, sigma)
print(h(dist(10000), 200), np.log2(dist.high - dist.low))

5.341523339158773
1.5371766422592845 1.5294468445267844
